<a href="https://colab.research.google.com/github/ruichenli2000/Data-Structure/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install gensim

In [2]:
! pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

In [3]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import gensim
from gensim.models.coherencemodel import CoherenceModel
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
import nltk


In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


ArrowInvalid: Not a Feather V1 or Arrow IPC file

In [8]:
C_R = pd.read_csv('/content/drive/MyDrive/colab_folder/Top100Wrestler.csv')


In [30]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=2f69510d33432d59a0a4e19aa7973e0fa47b340f229c122745f81c46698bd2a2
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [31]:
from langdetect import detect

In [32]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"
C_R["Language"] = C_R["Comment"].apply(detect_language)

C_R = C_R[C_R["Language"] == "en"].reset_index(drop=True)

In [33]:
ctfidf_model = ClassTfidfTransformer(
  reduce_frequent_words=True
)

In [34]:
topic_model = BERTopic(ctfidf_model=ctfidf_model)

topics, probs = topic_model.fit_transform(C_R['Comment'].to_list())


In [12]:
from joblib import load, dump


In [35]:
dump([topic_model, topics, probs], '/content/drive/MyDrive/colab_folder/topic_model.joblib')

['/content/drive/MyDrive/colab_folder/topic_model.joblib']

In [49]:
topic_model, topics, probs = load(
  '/content/drive/MyDrive/colab_folder/topic_model.joblib'
)

print(topic_model.get_topic_info().head(20))


    Topic  Count                                     Name  \
0      -1   1754                  -1_bryan_mic_promos_too   
1       0    244                     0_punk_cm_punks_2011   
2       1    240                    1_shes_she_her_female   
3       2    219                2_japanese_japan_njpw_mma   
4       3    213        3_undertaker_taker_deadman_streak   
5       4    211        4_jericho_chris_jerichos_reinvent   
6       5    191             5_smack_outfit_moves_swagger   
7       6    189               6_aj_styles_tna_phenomenal   
8       7    174       7_ospreay_ospreays_ricochet_flippy   
9       8    157  8_favorite_favourite_wrestlemania_loved   
10      9    156               9_kenny_omega_cleaner_bout   
11     10    151                    10_10_give_ten_rating   
12     11    126            11_shawn_michaels_rockers_hbk   
13     12    125      12_okada_okadas_tanahashi_rainmaker   
14     13    122           13_edge_christian_copeland_tlc   
15     14    110        

In [47]:
print(topic_model.get_topic(0))


[('he', 0.26574618160305696), ('his', 0.26489134674308573), ('to', 0.2554191237261478), ('and', 0.2540949546659485), ('of', 0.253148628012211), ('in', 0.25243457775982436), ('is', 0.24960170698479303), ('was', 0.24957648363181317), ('that', 0.2447656948574153), ('the', 0.24456826776703366)]


In [38]:
print(topic_model.get_document_info(C_R['Comment']))


                                               Document  Topic  \
0     John Bradshaw famously dubbed himself a wrestl...     61   
1     I personally slightly edge Misawa over Kobashi...     46   
2     If I'm being honest, Kobashi's probably one of...     46   
3     Known for his devastating chops and iconic mov...     61   
4     Kobashi was my first introduction to wrestling...     46   
...                                                 ...    ...   
9071  Good at everything, really. Quality matches, d...     -1   
9072  He is already one of the greatest independent ...    154   
9073  YouŽve got to see this guy live to belive how ...     -1   
9074  Chris is awesome! Best in the world, best in t...     56   
9075  Best Wrestler On The Planet... who's Danielson...     69   

                                       Name  \
0           61_kenta_kobashi_burning_hammer   
1           46_kobashi_misawa_cancer_hammer   
2           46_kobashi_misawa_cancer_hammer   
3           61_kent

In [39]:
print(topic_model.get_representative_docs(0))

['I\'m going to preface this by saying CM Punk is my favorite wrestler of ALL TIME. That being said, CM Punk is one of the last wrestlers who are actually the complete package. He can easily get people invested into what he\'s doing, he knows how to work the fans, he has a great understanding of in ring psychology as well as being able to give tribute to older wrestling with a modern twist to them. He has been apart of some of the greatest stories that AEW has ever seen, and throughout his career he has been able to make the most out of what was given to him. He\'s polarizing for a reason and he\'s a top 3 draw in professional wrestling today because of how good he is. His character work is damn near flawless, his ringwork is great, Punk just gets it. His match catalogue is amazing, I\'m going to try to go in order for listing what I consider to be his classic matches, His series with Chris Hero, His dog collar match with Raven, his trilogy with Samoa Joe, All of his matches against Da

In [40]:
print(topic_model.generate_topic_labels())

['-1_bryan_mic_promos', '0_punk_cm_punks', '1_shes_she_her', '2_japanese_japan_njpw', '3_undertaker_taker_deadman', '4_jericho_chris_jerichos', '5_smack_outfit_moves', '6_aj_styles_tna', '7_ospreay_ospreays_ricochet', '8_favorite_favourite_wrestlemania', '9_kenny_omega_cleaner', '10_10_give_ten', '11_shawn_michaels_rockers', '12_okada_okadas_tanahashi', '13_edge_christian_copeland', '14_tanahashi_tana_hiroshi', '15_asuka_shes_her', '16_kurt_angle_olympic', '17_flair_ric_flairs', '18_sting_crow_surfer', '19_austin_steve_cold', '20_gunther_gunthers_guntherwalter', '21_bret_hart_hitman', '22_misawa_mitsuharu_misawas', '23_aew_page_listened', '24_eddie_guerrero_eddies', '25_danielson_bryan_kofi', '26_heenan_manager_bobby', '27_ishii_tomohiro_pitbull', '28_foley_mick_mankind', '29_owens_kevin_zayn', '30_goat_vs_goats', '31_rock_hollywood_rocks', '32_naito_naitos_lij', '33_zayn_sami_underdog', '34_kota_ibushi_golden', '35_owen_brother_hart', '36_ad_undersized_mic', '37_tjpw_shoko_miyu', '38_

In [41]:
print(topic_model.reduce_topics(C_R['Comment'].to_list(), nr_topics=10))

BERTopic(calculate_probabilities=False, ctfidf_model=ClassTfidfTransformer(...), embedding_model=<bertopic.backend._sentencetransformers.SentenceTransformerBackend object at 0x7df527966dd0>, hdbscan_model=HDBSCAN(...), language=english, low_memory=False, min_topic_size=10, n_gram_range=(1, 1), nr_topics=10, representation_model=None, seed_topic_list=None, top_n_words=10, umap_model=UMAP(...), vectorizer_model=CountVectorizer(...), verbose=False, zeroshot_min_similarity=0.7, zeroshot_topic_list=None)
